# VOSS 2023 Project: Search for high-z QSO in imbalanced photometric samples

## Initialize Julia session
Install / load packages and change to directory containing the data.


In [1]:
using Pkg
if isnothing(Base.find_package("MyAstroUtils"))
    Pkg.add(url="https://github.com/gcalderone/MyAstroUtils.jl.git", rev="master")
    Pkg.add("Gnuplot")
    Pkg.add("AstroLib")
end

using MyAstroUtils, StatsBase, Random, AstroLib, Gnuplot
Gnuplot.options.mime[MIME"image/png"] = ""
Gnuplot.options.mime[MIME"image/svg+xml"] = ""
Gnuplot.options.mime[MIME"text/html"] = "svg enhanced standalone mouse size 900,450"
push!(Gnuplot.options.init, linetypes(:Set1_5, lw=3, ps=1.5))

cd("/data/VOSS23")

## Load subset_* catalogs and handle multiple match:

In [3]:
catGaia = fits2df("subset_Gaia_EDR3.fits.gz")
catPS = fits2df("subset_PanSTARRS1DR2.fits.gz");

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


Cross match Gaia vs PanSTARRS with a threshold of 5 arcsec:

In [4]:
jj = xmatch(catGaia.ra, catGaia.dec, catPS.raStack, catPS.decStack, 5.)

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:54


Input 1:      1828081 /      4043483  ( 45.21%)  -  max mult. 6
Input 2:      1705962 /      1709627  ( 99.79%)  -  max mult. 6
Output :      1900670


Plot histogram of distances.  Note that there are ambiguous matched sources (maximum multiplicity is 6).

In [28]:
@gp "set grid" ylog=true xlab="Match distance [arcsec]" ylab="Counts" :-
dist = gcirc.(2, catGaia[jj[1], :ra]     , catGaia[jj[1], :dec],
                 catPS[  jj[2], :raStack], catPS[  jj[2], :decStack])
hh = hist(dist, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'All match'"

Identify the best match among the previous ones and display the corresponding histogram of distances:

In [29]:
jjbest = best_match(catGaia.ra, catGaia.dec, catPS.raStack, catPS.decStack, jj)
display(jjbest)
dist = gcirc.(2, catGaia[jjbest[1], :ra]     , catGaia[jjbest[1], :dec], 
                 catPS[  jjbest[2], :raStack], catPS[  jjbest[2] , :decStack])
hh = hist(dist, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'Best match'"

Input 1:      1705225 /      4043483  ( 42.17%)  -  max mult. 1
Input 2:      1705225 /      1709627  ( 99.74%)  -  max mult. 1
Output :      1705225


Note that max. multiplicity is now 1, i.e. we removed all multiple matched source (except the best one).  However, the "best one", i.e. the match with smaller distance, is not necessary the best one hence it is necessary to drop all multiple match (also the best ones!) altoghether.

In [30]:
i = findall(countmatch(jj, 1) .== 1)
catGaia_singlematch = catGaia[i, :]
i = findall(countmatch(jj, 2) .== 1)
catPS_singlematch = catPS[i, :]
jj1 = xmatch(catGaia_singlematch.ra, catGaia_singlematch.dec, catPS_singlematch.raStack, catPS_singlematch.decStack, 5.)
display(jj1)
dist = gcirc.(2, catGaia_singlematch[jj1[1], :ra]     , catGaia_singlematch[jj1[1], :dec],
                 catPS_singlematch[  jj1[2], :raStack], catPS_singlematch[  jj1[2], :decStack])
hh = hist(dist, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'Single match'"

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:22


Input 1:      1519753 /      1756860  ( 86.50%)  -  max mult. 1
Input 2:      1519753 /      1522838  ( 99.80%)  -  max mult. 1
Output :      1519753


The green histogram lies slightly below the blue one, especially at distances between ~1 and 3.5 arcsec, i.e. we cleaned-up the list of matched source in the range where the probability for an ambiguous match is higher.

### Identify best matching thresholds

Replot previous histogram in linear scale:

In [32]:
@gp :- ylog=false xr=[0, 1.5]

The bulk of matched sources lies at distances <~0.2 arcsec, hence considering a conservative threshold of 0.5 arcsec between Gaia and PanSTARRS allows to identify reliable match.  Keep in mind that it is not possible to have a 100% clean sample here, we are happy with having it clean as much as possible.  Consider now the WISE catalog:

In [34]:
catWise = fits2df("subset_AllWise.fits.gz")
jj = xmatch(catGaia.ra, catGaia.dec, catWise.ra, catWise.dec, 5.)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
SortMerge 100%|█████████████████████████████████████████| Time: 0:00:44


Input 1:      1441239 /      4043483  ( 35.64%)  -  max mult. 2
Input 2:      1332188 /      1334038  ( 99.86%)  -  max mult. 6
Output :      1442366


Max. multiplicity is 6, hence we need to get rid of multiple match:

In [38]:
i = findall(countmatch(jj, 1) .== 1)
catGaia_singlematch = catGaia[i, :]
i = findall(countmatch(jj, 2) .== 1)
catWise_singlematch = catWise[i, :]
jj1 = xmatch(catGaia_singlematch.ra, catGaia_singlematch.dec, catWise_singlematch.ra, catWise_singlematch.dec, 2.)
display(jj1)
dist = gcirc.(2, catGaia_singlematch[jj1[1], :ra], catGaia_singlematch[jj1[1], :dec],
                 catWise_singlematch[jj1[2], :ra], catWise_singlematch[jj1[2], :dec])
hh = hist(dist, bs=0.1)
@gp hh.bins hh.counts "w histeps t 'Single match'"

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:05


Input 1:      1226806 /      1440112  ( 85.19%)  -  max mult. 1
Input 2:      1226806 /      1227137  ( 99.97%)  -  max mult. 1
Output :      1226806


We will adopt a conservative threshold is 0.75 arcsec between Gaia and Wise.

### Feature quick-look

Here we will quickly explore some of relevant quantities in the catalogs.  Let's start with the sky footprint:

In [47]:
@gp xlab="R.A. [deg]" ylab="Declination [deg]" hist(catGaia.ra, catGaia.dec, bs1=0.1, bs2=0.1)

Consider now the magnitudes in Gaia:

In [44]:
hh = hist(catGaia.phot_g_mean_mag, bs=0.5)
@gp    hh.bins hh.counts "w histeps t 'Gaia G'"
hh = hist(catGaia.phot_bp_mean_mag, bs=0.5)
@gp :- hh.bins hh.counts "w histeps t 'Gaia BP'"
hh = hist(catGaia.phot_rp_mean_mag, bs=0.5)
@gp :- hh.bins hh.counts "w histeps t 'Gaia RP'"

Magnitudes in Wise:

In [51]:
hh = hist(catWise.w1mpro, bs=0.1)
@gp    hh.bins hh.counts "w histeps t 'W1'"
hh = hist(catWise.w2mpro, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'W2'"
hh = hist(catWise.w3mpro, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'W3'"
hh = hist(catWise.w4mpro, bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'W4'"

Magnitudes in PanSTARRS:

In [53]:
hh = hist(catPS.gPSFMag, bs=0.5)
@gp     hh.bins hh.counts "w histeps t 'PS g'"
hh = hist(catPS.rPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS r'"
hh = hist(catPS.iPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS i'"
hh = hist(catPS.zPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS z'"
hh = hist(catPS.yPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS Y'"

**WARNING**: there are negative values in the columns!  We must get rid of such numbers

In [54]:
for col in [:gPSFMag, :rPSFMag, :iPSFMag, :zPSFMag, :yPSFMag]
    catPS[catPS[:, col] .< 0, col] .= NaN
end

Replot the histograms:

In [55]:
hh = hist(catPS.gPSFMag, bs=0.5)
@gp     hh.bins hh.counts "w histeps t 'PS g'"
hh = hist(catPS.rPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS r'"
hh = hist(catPS.iPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS i'"
hh = hist(catPS.zPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS z'"
hh = hist(catPS.yPSFMag, bs=0.5)
@gp :-  hh.bins hh.counts "w histeps t 'PS Y'"

The subset_* catalog are no longer needed, we will free some RAM before proceeding.

In [58]:
catGaia = nothing
catGaia_singlematch = nothing
catPS = nothing
catPS_singlematch = nothing
catWise = nothing
catWise_singlematch = nothing
GC.gc()

## Load main catalogs

In [2]:
main = fits2df("VOSS_classifications.fits.gz");

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


Keep only relevant columns:

In [3]:
main = main[:, [:qid, :RAd, :DECd, :objtype, :z_spec]];

Check object types: 

In [4]:
countmap(main.objtype)

Dict{String, Int64} with 8 entries:
  "Type2"     => 1992
  "EmLines"   => 28
  " "         => 7639184
  "QSO"       => 18177
  "Uncertain" => 5
  "Galaxy"    => 19094
  "Extended?" => 131487
  "BLLac"     => 147

...and redshift distribution

In [5]:
@gp xlab="Redshift" hist(main.z_spec, bs=0.2) ylog=true

Load photometric catalogs:

In [6]:
gaia = fits2df("Gaia_EDR3.fits.gz")
pans = fits2df("PanSTARRS1DR2.fits.gz")
wise = fits2df("AllWise.fits.gz")

# Remove negative values in PanSTARRS columns
for col in [:gPSFMag, :rPSFMag, :iPSFMag, :zPSFMag, :yPSFMag]
    pans[pans[:, col] .< 0, col] .= NaN
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03
Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


### Cross-matching
The `main` catalog is a collection of data from many catalogs, and the coordinates are taken also from Gaia, WISE and PanSTARSS, hence the matching distance may be zero in some case.  As a consequence we don't know in advance where do the coordinates in `main` comes from, hence we are going to use the largest matching distance: 0.75 arcsec.

In [7]:
jj = xmatch(main.RAd, main.DECd, gaia.ra, gaia.dec, 0.75)

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:31


Input 1:      7804472 /      7810114  ( 99.93%)  -  max mult. 2
Input 2:      7804472 /      7804642  (100.00%)  -  max mult. 2
Output :      7804473


With 0.75 we have ambiguous match, try again with 0.65:

In [9]:
jj = xmatch(main.RAd, main.DECd, gaia.ra, gaia.dec, 0.65)

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:27


Input 1:      7770060 /      7810114  ( 99.49%)  -  max mult. 1
Input 2:      7770060 /      7804642  ( 99.56%)  -  max mult. 1
Output :      7770060


Great! Multiplicity is now 1 in both directions, we can join the catalogs.  Note that we discard a few columns which are not necessary to avoid conflicts when adding further catalogs, and to save memory. If necessary, we may recover discarded columns in a later stage.

In [10]:
main[!, :gaia_id] .= 0
main[jj[1], :gaia_id] .= gaia[jj[2], :_id]
tmp = select(gaia[jj[2], :], :_id => :gaia_id, Not([:_id, :ra, :dec, :ra_error, :dec_error, :source_id,
                                                    :phot_g_mean_flux , :phot_g_mean_flux_error, 
                                                    :phot_bp_mean_flux, :phot_bp_mean_flux_error, 
                                                    :phot_rp_mean_flux, :phot_rp_mean_flux_error]))
leftjoin!(main, tmp, on=:gaia_id)
@assert all(main[jj[1], :gaia_id] .== gaia[jj[2], :_id])
gaia = nothing
tmp = nothing
GC.gc()

Let's match Wise catalog:

In [11]:
jj = xmatch(main.RAd, main.DECd, wise.ra, wise.dec, 0.75)

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:26


Input 1:      6515767 /      7810114  ( 83.43%)  -  max mult. 1
Input 2:      6515767 /      6515767  (100.00%)  -  max mult. 1
Output :      6515767


No multiple match, join the catalogs:

In [12]:
main[!, :wise_id] .= 0
main[jj[1], :wise_id] .= wise[jj[2], :_id]
tmp = select(wise[jj[2], :], :_id => :wise_id, Not([:_id, :ra, :dec, :source_id,
                                                    :w1sigmpro, :w2sigmpro, :w3sigmpro, :w4sigmpro,
                                                    :w1snr, :w2snr, :w3snr, :w4snr, 
                                                    :pmra, :sigpmra, :pmdec, :sigpmdec,
                                                    :ext_flg, :var_flg]))
leftjoin!(main, tmp, on=:wise_id)
@assert all(main[jj[1], :wise_id] .== wise[jj[2], :_id])
wise = nothing
tmp = nothing
GC.gc()

Proceed with PanSTARRS:

In [13]:
jj = xmatch(main.RAd, main.DECd, pans.raStack, pans.decStack, 0.75)

SortMerge 100%|█████████████████████████████████████████| Time: 0:00:31


Input 1:      7809641 /      7810114  ( 99.99%)  -  max mult. 1
Input 2:      7809641 /      7810114  ( 99.99%)  -  max mult. 1
Output :      7809641


In [14]:
main[!, :pans_id] .= 0
main[jj[1], :pans_id] .= pans[jj[2], :_id]
tmp = select(pans[jj[2], :], :_id => :pans_id, Not([:_id, :raStack, :decStack, 
                                                    :objName, :objID, :objInfoFlag, :qualityFlag,
                                                    :gPSFMagErr, :rPSFMagErr, :iPSFMagErr, :zPSFMagErr, :yPSFMagErr]))
leftjoin!(main, tmp, on=:pans_id)
@assert all(main[jj[1], :pans_id] .== pans[jj[2], :_id])
pans = nothing
tmp = nothing
GC.gc()

The final "*big table*" looks as follows:

In [15]:
main

Row,qid,RAd,DECd,objtype,z_spec,gaia_id,pmra,pmdec,pm,pmra_error,pmdec_error,parallax,parallax_error,parallax_over_error,phot_bp_mean_mag,phot_rp_mean_mag,phot_g_mean_mag,wise_id,w1mpro,w2mpro,w3mpro,w4mpro,pans_id,gPSFMag,rPSFMag,iPSFMag,zPSFMag,yPSFMag
,Int64,Float64,Float64,String,Float32,Int64,Float64?,Float64?,Float32?,Float32?,Float32?,Float64?,Float32?,Float32?,Float32?,Float32?,Float32?,Int64,Float32?,Float32?,Float32?,Float32?,Int64,Float64?,Float64?,Float64?,Float64?,Float64?
1,20980005,309.628,-25.4388,,NaN,14074140,2.06038,-2.12561,2.9603,0.0718607,0.0624142,0.834332,0.0712241,11.7142,17.7311,16.3309,17.0846,735856251,14.696,14.762,12.441,8.846,18675517,17.9579,17.055,16.6996,16.6098,16.337
2,28443138,358.835,-21.1004,,NaN,23419137,29.6826,-16.4506,33.9364,0.159776,0.12759,1.45507,0.136717,10.643,19.0752,17.027,18.0174,249299010,14.804,14.763,12.573,9.046,26259440,19.3526,18.1784,17.4013,17.0359,16.8631
3,6971647,170.511,-18.809,,NaN,7787223,-16.4497,0.428132,16.4552,0.0440459,0.0330324,0.526193,0.043636,12.0587,16.3822,15.5985,16.0783,366699122,14.608,14.671,12.303,8.861,4365788,16.6159,16.0422,15.8902,15.8185,15.8125
4,30839426,59.0739,10.7332,,NaN,117500651,0.380582,-3.69964,3.71916,0.0690588,0.049467,0.258207,0.0677148,3.81315,17.0005,15.9675,16.5673,340034728,14.64,14.618,12.504,8.588,28747168,17.0372,16.5273,16.3547,16.179,16.1801
5,3417102,138.714,-8.49102,,NaN,29802627,-2.62356,-4.68316,5.36796,0.155761,0.129209,0.307143,0.169975,1.80698,19.1639,17.7185,18.479,611163634,15.867,16.142,12.329,9.009,722215,19.235,18.4693,18.057,17.9065,17.8279
6,6457883,166.314,-9.61365,,NaN,12802919,-8.98555,-2.75473,9.39833,0.0806062,0.067881,0.204979,0.0700321,2.92693,17.229,16.4216,16.9181,385485715,15.454,15.354,12.609,8.859,3836957,17.273,16.8681,16.8162,16.7193,16.678
7,27629799,345.83,9.27746,,NaN,121845374,3.4095,-1.73856,3.82718,0.286371,0.19741,0.490793,0.254316,1.92986,19.167,17.9937,18.6647,280450429,16.191,16.224,12.548,8.691,25424113,19.2866,18.6055,18.3138,18.214,18.1791
8,17942755,245.454,2.25222,,NaN,125186344,-6.10335,-5.13964,7.97915,0.47309,0.316799,0.563325,0.437785,1.28676,20.9211,18.4056,19.5866,0,missing,missing,missing,missing,15592908,21.5365,20.2049,18.877,18.2483,17.9962
9,14136139,225.344,-15.2747,,NaN,43699035,4.68716,-26.54,26.9507,0.373605,0.343796,1.87328,0.330209,5.67301,20.8573,17.9134,19.1377,683302406,15.281,15.071,12.563,9.004,11732859,20.9971,19.734,18.4006,17.7756,17.5341


And the number of rows is:

In [16]:
nrow(main)

7810114

### Identify stars

Check distributions of parallax and proper motions (in units of the associated uncertainties) for the QSOs and for the whole sample:

In [17]:
@gp title="Parallax" xr=[-4,4] "set grid" ylog=true
hh = hist(main[:, :parallax_over_error] .* 1., bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'All'"
i = findall((main.objtype .== "QSO")  .&  (main.z_spec .> 2.5))
hh = hist(main[i, :parallax_over_error], bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'high-z QSO'"

[ Info: Neglecting missing values (40054)
[ Info: Neglecting missing values (13)


In [18]:
@gp title="Proper motion (R.A.)" xr=[-4,4] "set grid" ylog=true
hh = hist(main[:, :pmra] ./ main[:, :pmra_error], bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'All'"
hh = hist(main[i, :pmra] ./ main[i, :pmra_error], bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'high-z QSO'"

[ Info: Neglecting missing values (40054)
[ Info: Neglecting missing values (13)


In [19]:
@gp title="Proper motion (Decl.)" xr=[-4,4] "set grid" ylog=true
hh = hist(main[:, :pmdec] ./ main[:, :pmdec_error], bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'All'"
i = findall((main.objtype .== "QSO")  .&  (main.z_spec .> 2.5))
hh = hist(main[i, :pmdec] ./ main[i, :pmdec_error], bs=0.1)
@gp :- hh.bins hh.counts "w histeps t 'high-z QSO'"

[ Info: Neglecting missing values (40054)
[ Info: Neglecting missing values (13)


Check high-z QSO which would be misclassified as stars using cuts:

In [20]:
i = findall((main.objtype .== "QSO")  .&  (main.z_spec .> 2.5)  .&
            ((.!ismissing.(main.parallax_over_error)  .&   (abs.(main.parallax_over_error)       .>= 3.4)) .|
             (.!ismissing.(main.pmra)                 .&   (abs.(main.pmra  ./ main.pmra_error)  .>= 3.2)) .|
             (.!ismissing.(main.pmdec)                .&   (abs.(main.pmdec ./ main.pmdec_error) .>= 3.2))))
tmp = main[i, [:qid, :RAd, :DECd, :objtype, :z_spec, :parallax_over_error]]
tmp[!, :pmra_over_error]  .= main[i, :pmra]  ./ main[i, :pmra_error]
tmp[!, :pmdec_over_error] .= main[i, :pmdec] ./ main[i, :pmdec_error]
sort!(tmp, :z_spec)

Row,qid,RAd,DECd,objtype,z_spec,parallax_over_error,pmra_over_error,pmdec_over_error
,Int64,Float64,Float64,String,Float32,Float32?,Float64,Float64
1,18189003,244.132,13.866,QSO,2.53403,-1.50946,-0.416791,-3.92867
2,1517336,36.0202,1.82833,QSO,2.72075,-5.77852,4.04208,6.7212
3,2224363,234.68,8.71028,QSO,2.735,97.124,130.762,-582.172
4,848344,51.2534,-21.6781,QSO,2.828,-0.6606,1.31959,-3.66936
5,1831505,219.598,8.52463,QSO,2.838,0.735483,3.29687,0.0811445
6,1187665,334.698,-1.2124,QSO,2.8815,-0.264689,3.88509,2.74651
7,2135171,317.446,6.94304,QSO,2.94001,4.27243,7.6251,-7.34463
8,1040074,12.8418,-20.837,QSO,3.13616,-0.662081,-3.27515,-0.271866
9,1136718,13.9271,-6.23667,QSO,3.39,0.087265,-2.22788,-3.22498


The above sources requires attention (especially the last one!), it may be worth checking an online atlas...

Now flag the stars (only if no previous classification is available):

In [21]:
i = findall((main.objtype .== " ")  .&
            ((.!ismissing.(main.parallax_over_error)  .&   (abs.(main.parallax_over_error)       .>= 3.4)) .|
             (.!ismissing.(main.pmra)                 .&   (abs.(main.pmra  ./ main.pmra_error)  .>= 3.2)) .|
             (.!ismissing.(main.pmdec)                .&   (abs.(main.pmdec ./ main.pmdec_error) .>= 3.2))))
main[i, :objtype] .= "Star";

Narrow down labels:

In [22]:
main[main.objtype .== " ", :objtype] .= ""
main[main.objtype .== "Extended?", :objtype] .= "Galaxy"
main[(main.objtype .== "QSO")  .&  (main.z_spec .> 2.5), :objtype] .= "QSOhighZ"
main[(main.objtype .== "QSO")  .&  (main.z_spec .<=2.5), :objtype] .= "QSOlowZ"
i = findall((main.objtype .!= "QSOhighZ")  .&
            (main.objtype .!= "QSOlowZ")   .&
            (main.objtype .!= "Galaxy")    .&
            (main.objtype .!= "Star")      .&
            (main.objtype .!= ""))
main[i, :objtype] .= "other"
        
countmap(main.objtype)

Dict{String, Int64} with 6 entries:
  "other"    => 2172
  "QSOlowZ"  => 16274
  ""         => 1678433
  "Star"     => 5960751
  "Galaxy"   => 150581
  "QSOhighZ" => 1903

### Save the big table in a FITS file

In [ ]:
f = FITS("VOSS23_BigTable.fits", "w")
write(f, main)
close(f)